In [92]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
pd.set_option('display.max_columns',None)



from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(15,5)
%config InlineBackend.figure_format = 'png'
%matplotlib inline

Загружаем данные в датафреймы 

In [93]:
df_ost = pd.read_pickle("./df_ost.pkl")

In [94]:
df_category = pd.read_pickle("./df_category.pkl")

In [95]:
#Исправляем табличку 
df_category.columns = list(map(''.join, df_category.columns.values))

In [96]:
df_category

,id_group,name_gr,id_group2,name_gr2,id_tov,name_tov,date_ttcount
0,11,Прочее,100,ХозТовары,942,Комус Ёмкость РК-19,6
1,11,Прочее,100,ХозТовары,1260,Кассовая лента ВкусВилл (широкая),26
2,11,Прочее,100,ХозТовары,1269,Зубочистки (1000шт),2
3,11,Прочее,100,ХозТовары,1271,Стаканчики (100шт),3
4,11,Прочее,100,ХозТовары,1272,Скотч,5
...,...,...,...,...,...,...,...
6640,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,30085,"Кофе зерновой BLEND #5, 1кг",71
6641,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,30572,Кофе Флэт Уайт 0.2,6
6642,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,39254,"Какао-напиток ""Choco 01 Rich Dark"", 1кг",141
6643,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,39255,"Кофе зерновой ""SOLD OUT"", 1 кг",141


In [97]:
popular_id_tov = df_category['id_tov'][(df_category['name_gr'] == 'Молочная Продукция') 
&(df_category['date_ttcount'] > 3000)&(7000 > df_category['date_ttcount'])].values.tolist()

In [98]:
df_worst_milk_sales_bad_shops = pd.read_pickle("./df_worst_milk_sales_bad_shops.pkl")

In [99]:
df_best_milk_sales_good_shops = pd.read_pickle("./df_best_milk_sales_good_shops.pkl")

In [100]:

lis_good_shops = df_best_milk_sales_good_shops.index.values.tolist()
lis_bad_shops = df_worst_milk_sales_bad_shops.index.values.tolist()

In [101]:
len(lis_good_shops)

15

In [102]:
len(lis_bad_shops)

19

In [104]:
cat = 'Молочная Продукция'
group_type = 'name_gr'

bad_shop_oos_stock = df_ost[(df_ost[group_type] == cat) 
& (df_ost['evening'].isna()) 
& (df_ost['id_tt'].isin(lis_bad_shops))
& (df_ost['id_tov'].isin(popular_id_tov))]

good_shop_oos_stock = df_ost[(df_ost[group_type] == cat) 
& (df_ost['evening'].isna()) 
& (df_ost['id_tt'].isin(lis_good_shops))
& (df_ost['id_tov'].isin(popular_id_tov))]

print('OOS in good shops :',len(good_shop_oos_stock.index)*19//15)
print('OOS in bad shops :',len(bad_shop_oos_stock.index))
print('OOS in bad shops percent :',1- len(good_shop_oos_stock.index)/len(bad_shop_oos_stock.index))



OOS in good shops : 15374
OOS in bad shops : 26314
OOS in bad shops percent : 0.5387246332750627


In [105]:
cat = 'Молоко'
group_type = 'name_gr2'

def OOS(cat,group_type):
    bad_shop_oos_stock = df_ost[(df_ost[group_type] == cat) 
    & (df_ost['evening'].isna()) 
    & (df_ost['id_tt'].isin(lis_bad_shops))
    & (df_ost['id_tov'].isin(popular_id_tov))]

    good_shop_oos_stock = df_ost[(df_ost[group_type] == cat) 
    & (df_ost['evening'].isna()) 
    & (df_ost['id_tt'].isin(lis_good_shops))
    & (df_ost['id_tov'].isin(popular_id_tov))]

    OOS_in_good_shops  = len(good_shop_oos_stock.index)*19//15
    OOS_in_bad_shops  = len(bad_shop_oos_stock.index)
    OOS_percent = 1- len(good_shop_oos_stock.index)/(len(bad_shop_oos_stock.index)+1)

    return OOS_in_good_shops, OOS_in_bad_shops , OOS_percent


In [106]:
df_OOS = pd.DataFrame(columns = ['name_gr2','OOS_in_good_shops','OOS_in_bad_shops','OOS_percent'])

In [107]:
gr2 = ['Молоко','Творога мягкие','Сметана','Кумыс','Сливки','Кисломолочные продукты','Продукты из козьего молока','Детское питьевое молочное питание','Йогурты ложковые','Масло','Молочные напитки','Растительные ложковые продукты, сырки','Йогурты питьевые','Творог классика','Сырки творожные','Детское ложковое молочное питание','Лакомства молочные','Растительные питьевые продукты','Кефир','Безлактозные продукты'] 

In [108]:
for gr in gr2:
    OOS_in_good_shops ,  OOS_in_bad_shops , OOS_percent = OOS(gr ,'name_gr2')
    new_row = {'name_gr2':gr, 'OOS_in_good_shops':OOS_in_good_shops, 'OOS_in_bad_shops':OOS_in_bad_shops, 'OOS_percent':OOS_percent}
    df_OOS = df_OOS.append(new_row, ignore_index=True)


C:\Temp\ipykernel_236\2481408390.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OOS = df_OOS.append(new_row, ignore_index=True)
C:\Temp\ipykernel_236\2481408390.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OOS = df_OOS.append(new_row, ignore_index=True)
C:\Temp\ipykernel_236\2481408390.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OOS = df_OOS.append(new_row, ignore_index=True)
C:\Temp\ipykernel_236\2481408390.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OOS = df_OOS.append(new_row, ignore_index=True)
C:\Temp\ipykernel_236\2481408390.py:4: FutureWarning: The frame.append method is deprecated and will

In [109]:
df_OOS

,name_gr2,OOS_in_good_shops,OOS_in_bad_shops,OOS_percent
0,Молоко,1274,1910,0.473574
1,Творога мягкие,1801,3332,0.573357
2,Сметана,580,1128,0.594331
3,Кумыс,0,0,1.0
4,Сливки,658,1198,0.566305
5,Кисломолочные продукты,762,1166,0.484147
6,Продукты из козьего молока,0,0,1.0
7,Детское питьевое молочное питание,1373,2270,0.522677
8,Йогурты ложковые,1398,2628,0.580068
9,Масло,167,190,0.308901


In [111]:
df_category_milk = df_category[df_category['name_gr'] =='Молочная Продукция' ]

In [127]:
df_category_milk2 = df_category_milk[['name_gr2','date_ttcount']]

In [128]:
df_category_milk2.reset_index(drop = True, inplace = True)


In [129]:
df_category_milk2

,name_gr2,date_ttcount
0,Молоко,5317
1,Молоко,4349
2,Молоко,5139
3,Молоко,5373
4,Молоко,5390
...,...,...
346,Безлактозные продукты,1097
347,Безлактозные продукты,2928
348,Безлактозные продукты,1334
349,Безлактозные продукты,1815


In [135]:
df_category_milk3 = df_category_milk2.groupby(['name_gr2']).agg({'date_ttcount': ['sum']}).reset_index()

In [138]:
#Исправляем табличку 
df_category_milk3.columns = list(map(''.join, df_category_milk3.columns.values))

In [139]:
df_category_milk3

,name_gr2,date_ttcountsum
0,Безлактозные продукты,20516
1,Детское ложковое молочное питание,39986
2,Детское питьевое молочное питание,94324
3,Йогурты ложковые,60413
4,Йогурты питьевые,50997
5,Кефир,53863
6,Кисломолочные продукты,34406
7,Кумыс,501
8,Лакомства молочные,26264
9,Масло,27355
